# Time Basset Parts

Estimate how much time is spent in convolutional vs fc layers for Basset.

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.__version__

'2.3.0'

In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
device = 'GPU:0' if tf.config.experimental.list_physical_devices('GPU') else '/device:CPU:0'
device

'GPU:0'

## Conv Layers

In [5]:
inp = tf.keras.Input(shape=(1000, 4))

# conv mxp 1
x = tf.keras.layers.Conv1D(
    300, 19, strides=1, padding='same', activation='relu', name='conv1')(inp)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool1D(3)(x)

# conv mxp 2
x = tf.keras.layers.Conv1D(
    200, 11, strides=1, padding='same', activation='relu', name='conv2')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool1D(4)(x)

# conv mxp 3
x = tf.keras.layers.Conv1D(
    200, 7, strides=1, padding='same', activation='relu', name='conv3')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool1D(4)(x)

# fc
x = tf.keras.layers.Flatten()(x)

# sum it up
x = tf.keras.backend.sum(x, axis= -1, keepdims=True)

conv_model = tf.keras.Model(inp, x)

In [6]:
x = tf.constant(np.random.random((512,1000,4)))*1
x.device 

'/job:localhost/replica:0/task:0/device:GPU:0'

In [7]:
# dry run
conv_model(x[:10], training=False)

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[166.94147],
       [167.26965],
       [165.38687],
       [163.8162 ],
       [165.77713],
       [166.21245],
       [166.28622],
       [164.19704],
       [167.00418],
       [165.78748]], dtype=float32)>

In [9]:
%timeit conv_model(x, training=False)

75.9 ms ± 198 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## FC Layers

In [10]:
inp = tf.keras.Input(shape=(4000))

x = tf.keras.layers.Dense(1000, activation='relu', name='fc1')(inp)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(1000, activation='relu', name='fc2')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(1, name='fc3')(x)

fc_model = tf.keras.Model(inputs=inp, outputs=x)

In [11]:
x = tf.constant(np.random.random((512, 4000)))*1
x.device 

'/job:localhost/replica:0/task:0/device:GPU:0'

In [12]:
# dry run
fc_model(x[:10], training=False)

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[0.07535696],
       [0.39930308],
       [0.37391436],
       [0.2166034 ],
       [0.29246455],
       [0.4637056 ],
       [0.27540737],
       [0.35666835],
       [0.26806518],
       [0.40557113]], dtype=float32)>

In [13]:
%timeit fc_model(x, training=False)

1.91 ms ± 120 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Full Model

In [15]:
inp = tf.keras.Input(shape=(1000, 4))

# conv mxp 1
x = tf.keras.layers.Conv1D(
    300, 19, strides=1, padding='same', activation='relu', name='conv1')(inp)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool1D(3)(x)

# conv mxp 2
x = tf.keras.layers.Conv1D(
    200, 11, strides=1, padding='same', activation='relu', name='conv2')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool1D(4)(x)

# conv mxp 3
x = tf.keras.layers.Conv1D(
    200, 7, strides=1, padding='same', activation='relu', name='conv3')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool1D(4)(x)

# fc
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1000, activation='relu', name='fc1')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(1000, activation='relu', name='fc2')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(1, name='fc3')(x)


full_model = tf.keras.Model(inp, x)

In [17]:
x = tf.constant(np.random.random((512,1000,4)))*1
x.device 

'/job:localhost/replica:0/task:0/device:GPU:0'

In [18]:
# dry run
full_model(x[:10], training=False)

<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
array([[0.11044829],
       [0.10558513],
       [0.10426483],
       [0.09787206],
       [0.12151661],
       [0.12320254],
       [0.11397058],
       [0.11834613],
       [0.1088988 ],
       [0.10269587]], dtype=float32)>

In [19]:
%timeit full_model(x, training=False)

77.2 ms ± 66.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
